In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
artworks = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv')
artworks.columns

Index(['Title', 'Artist', 'ConstituentID', 'ArtistBio', 'Nationality',
       'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'Dimensions',
       'CreditLine', 'AccessionNumber', 'Classification', 'Department',
       'DateAcquired', 'Cataloged', 'ObjectID', 'URL', 'ThumbnailURL',
       'Circumference (cm)', 'Depth (cm)', 'Diameter (cm)', 'Height (cm)',
       'Length (cm)', 'Weight (kg)', 'Width (cm)', 'Seat Height (cm)',
       'Duration (sec.)'],
      dtype='object')

In [3]:
#preprocessing/cleaning

#select columns of interest
artworks = artworks[['Artist', 'Nationality', 'Gender', 'Date', 'Department',
                     'DateAcquired', 'URL', 'ThumbnailURL', 'Height (cm)',
                     'Width (cm)']]

#convert urls to booleans indicating if they are present
artworks['URL'] = artworks['URL'].notnull()
artworks['ThumbnailURL'] = artworks['ThumbnailURL'].notnull()

#drop films and other tricky rows
artworks = artworks[artworks['Department'] != 'Film']
artworks = artworks[artworks['Department'] != 'Media and Performance Art']
artworks = artworks[artworks['Department'] != 'Fluxus Collection']

#drop missing data
artworks = artworks.dropna()

artworks.head()

,Artist,Nationality,Gender,Date,Department,DateAcquired,URL,ThumbnailURL,Height (cm),Width (cm)
0,Otto Wagner,(Austrian),(Male),1896,Architecture & Design,1996-04-09,True,True,48.6000,168.9000
1,Christian de Portzamparc,(French),(Male),1987,Architecture & Design,1995-01-17,True,True,40.6401,29.8451
2,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,34.3000,31.8000
3,Bernard Tschumi,(),(Male),1980,Architecture & Design,1995-01-17,True,True,50.8000,50.8000
4,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,38.4000,19.1000


## Building a model
Use multi-layer perceptron modeling (MLP) to classify the department a piece should go into using everything except the department name

In [4]:
#check data types before feeding to sklearn MLP
artworks.dtypes

Artist           object
Nationality      object
Gender           object
Date             object
Department       object
DateAcquired     object
URL                bool
ThumbnailURL       bool
Height (cm)     float64
Width (cm)      float64
dtype: object

In [5]:
#transform DateAcquired from object to datetime object
artworks['DateAcquired'] = pd.to_datetime(artworks.DateAcquired)

#add feature for just the year artwork acquired
artworks['YearAcquired'] = artworks.DateAcquired.dt.year
artworks['YearAcquired'].dtype

dtype('int64')

In [7]:
#more misc cleaning
#remove multiple nationalities, genders, artists
artworks.loc[artworks['Gender'].str.contains('\) \('), 'Gender'] = '\(multiple_persons)'
artworks.loc[artworks['Nationality'].str.contains('\) \('), 'Nationality'] = '\(multiple_nationalities\)'
artworks.loc[artworks['Artist'].str.contains(','), 'Artist'] = 'Multiple_Artists'

#convert dates to start date, cuts down number of distinct examples
artworks['Date'] = pd.Series(artworks.Date.str.extract(
    '([0-9]{4})', expand=False))[:-1]

#final column/NA drops
X = artworks.drop(['Department', 'DateAcquired', 'Artist', 'Nationality', 'Date'], 1)

#create dummies separately
artists = pd.get_dummies(artworks.Artist)
nationalities = pd.get_dummies(artworks.Nationality)
dates = pd.get_dummies(artworks.Date)

#concat with other variables (artists left out for now since it slows everything down)
X = pd.get_dummies(X, sparse=True)
X = pd.concat([X, nationalities, dates], axis=1)

#set target
y = artworks.Department

In [8]:
#modeling time, NNs are very computationally intensive and may take a few minutes
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(1000,))
mlp.fit(X,y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [9]:
mlp.score(X,y)

0.6808563155646271

In [10]:
y.value_counts()/len(y)

Prints & Illustrated Books    0.523140
Photography                   0.228737
Architecture & Design         0.111131
Drawings                      0.103081
Painting & Sculpture          0.033911
Name: Department, dtype: float64

In [11]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp, X, y, cv=5)

array([0.61146708, 0.61614771, 0.55802915, 0.48400266, 0.40925706])

**Results:**
- Model is overfitting
    - NNs that aren't given enough data for the number of features tend to overfit
    - (In general) NNs like *a lot* of data
- NNs can take a long time to run
    - Increase layer size -> increase run time
- Adding artists booleans to this model would exacerbate both the overfitting & runtime issues

## Model parameters

- hidden_layer_sizes: how many and how large to make layers
    - (1000,): 1000 neurons wide, one layer (this model)
    - (100, 4,): two layers, one 100 neurons wide, the other 4 neurons wide
    - Choosing layer size:
        - Determined by computational resources and cross validation
        - Generally less than the number of input variables
- alpha: scales regularization parameter penalty for large coefficients (just like advanced regression models)
- activation: function that determines whether the output from an individual perceptron is binary or continuous
    - relu: default, rectified linear unit function, binary
    - logistic: sklearn terminology for logistic sigmoid function, allows for continuous variables ranging 0 to 1


## Drill: playing with layers
Experiment with different hidden layer structures, using a subset of the data to improve runtime. See how things vary between layer structures. Can also play with other parameters & feature selection